In [3]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import pandas as pd
import pydst
dst = pydst.Dst(lang='en')

In [4]:
#Ultimately, we want to look at the share of people who have a bachelor's degree or higher across municipalities and time
#So we want the table HFUDD10
dst.get_data(table_id = 'HFUDD10')

,BOPOMR,HERKOMST,HFUDD,ALDER,KØN,TID,INDHOLD
0,All Denmark,Total,Total,"Age, total",Total,2006,3833359


In [5]:
#Now, we take a look at the variabels
hfudd_var = dst.get_variables(table_id='HFUDD10')
hfudd_var

,id,text,elimination,time,map,values
0,BOPOMR,region,True,False,denmark_municipality_07,"[{'id': '000', 'text': 'All Denmark'}, {'id': ..."
1,HERKOMST,ancestry,True,False,NaN,"[{'id': 'TOT', 'text': 'Total'}, {'id': '5', '..."
2,HFUDD,highest education completed,True,False,NaN,"[{'id': 'TOT', 'text': 'Total'}, {'id': 'H10',..."
3,ALDER,age,True,False,NaN,"[{'id': 'TOT', 'text': 'Age, total'}, {'id': '..."
4,KØN,sex,True,False,NaN,"[{'id': 'TOT', 'text': 'Total'}, {'id': 'M', '..."
5,Tid,time,False,True,NaN,"[{'id': '2006', 'text': '2006'}, {'id': '2007'..."


In [6]:
#Next, look at the dictionary under values, to see the categories in order to specify what variables we need
udd = hfudd_var.loc[2,'values']
udd

[{'id': 'TOT', 'text': 'Total'},
 {'id': 'H10', 'text': 'H10 Primary education'},
 {'id': 'H1001', 'text': 'H1001 No education'},
 {'id': 'H1010', 'text': 'H1010 Primary school through to 6th grade'},
 {'id': 'H1020', 'text': 'H1020 Primary school 7th-9th grade'},
 {'id': 'H1030', 'text': 'H1030 Primary school 10th grade'},
 {'id': 'H20', 'text': 'H20 Upper secondary education'},
 {'id': 'H2010',
  'text': 'H2010 Upper secondary education, General (stx, hf, student courses)'},
 {'id': 'H2020',
  'text': 'H2020 Upper secondary education, General (hhx, htx)'},
 {'id': 'H2030', 'text': 'H2030 International upper secondary education'},
 {'id': 'H30', 'text': 'H30 Vocational Education and Training (VET)'},
 {'id': 'H3010', 'text': 'H3010 Care, health and education (OSP)'},
 {'id': 'H3015',
  'text': 'H3015 Office, commercial and business services (KHF)'},
 {'id': 'H3020', 'text': 'H3020 Food etc. (FJO)'},
 {'id': 'H3025', 'text': 'H3025 Agriculture and nature (FJO)'},
 {'id': 'H3030', 'text

In [7]:
#First, we create list of the main categories
# Next we pick them from the HFUDD10 table and put into the dataframe hfudd10
hfudd_list = ['TOT','H60','H70','H80']
hfudd10 = dst.get_data(table_id = 'HFUDD10', variables={'BOPOMR':['*'], 'HFUDD':hfudd_list, 'TID':['*']})
hfudd10.tail()

,BOPOMR,HFUDD,TID,HERKOMST,ALDER,KØN,INDHOLD
5839,Vesthimmerlands,H60 Bachelors programmes,2019,Total,"Age, total",Total,125
5840,Mariagerfjord,H80 PhD programmes,2019,Total,"Age, total",Total,65
5841,Vesthimmerlands,H70 Masters programmes,2019,Total,"Age, total",Total,767
5842,Vesthimmerlands,H80 PhD programmes,2019,Total,"Age, total",Total,43
5843,Mariagerfjord,Total,2019,Total,"Age, total",Total,28720


In [8]:
#Drop unnecessary columns
hfudd10.drop(['ALDER','HERKOMST','KØN'], axis=1, inplace=True)

In [10]:
#Delete all rows that is not a municipality in the residence column
for val in ['Region', 'Province', 'All Denmark']: 
    I = hfudd10.BOPOMR.str.contains(val)
    hfudd10 = hfudd10.loc[I == False]

In [11]:
#Select all of the higher education categories sum them across each municipality and year.
group = hfudd10.loc[(hfudd10['HFUDD'] != 'Total')].copy()
group1 = group.groupby(['BOPOMR', 'TID'])['INDHOLD'].sum()
group2 = pd.DataFrame(group1)
group2.rename(columns = {'INDHOLD':'HIGH_EDU'}, inplace=True) #Rename column

In [18]:
#Select the total in each municipality and year.
total = hfudd10.loc[(hfudd10['HFUDD'] == 'Total')].copy()
total1 = total.groupby(['BOPOMR', 'TID'])['INDHOLD'].sum()
total2 = pd.DataFrame(total1)
total2.rename(columns = {'INDHOLD':'TOTAL_EDU'}, inplace=True) #Rename column

In [19]:
#Merge the data, reset index and get some proper variable names
merge = pd.merge(group2,total2,how='left',on=['BOPOMR','TID'])
merge['HIGH_EDU_SHARE'] = merge['HIGH_EDU'] / merge['TOTAL_EDU'] * 100
merge.reset_index(inplace=True)
merge.rename(columns = {'BOPOMR':'MUNICIPALITY','TID':'YEAR'}, inplace=True) #Rename column
merge

,MUNICIPALITY,YEAR,HIGH_EDU,TOTAL_EDU,HIGH_EDU_SHARE
0,Aabenraa,2006,1322,41865,3.157769
1,Aabenraa,2007,1367,41938,3.259574
2,Aabenraa,2008,1449,42097,3.442051
3,Aabenraa,2009,1498,42255,3.545143
4,Aabenraa,2010,1515,41920,3.614027
...,...,...,...,...,...
1381,Ærø,2015,195,4119,4.734159
1382,Ærø,2016,205,4089,5.013451
1383,Ærø,2017,211,3937,5.359411
1384,Ærø,2018,230,3887,5.917160


In [22]:
#Ultimately, we want to look at the share of people who have a bachelor's degree or higher across municipalities and time
#So we want the table HFUDD10
dst.get_data(table_id = 'HFUDD10')

#Now, we take a look at the variabels
hfudd_var = dst.get_variables(table_id='HFUDD10')
hfudd_var

#Next, look at the dictionary under values, to see the categories in order to specify what variables we need
udd = hfudd_var.loc[2,'values']
udd

#First, we create a list of the main categories i.e. education level higher or equal to bachelor
# Next we pick them from the HFUDD10 table and put into the dataframe hfudd10
hfudd_list = ['TOT','H60','H70','H80']
hfudd10 = dst.get_data(table_id = 'HFUDD10', variables={'BOPOMR':['*'], 'HFUDD':hfudd_list, 'TID':['*']})
hfudd10.tail()

#Drop unnecessary columns
hfudd10.drop(['ALDER','HERKOMST','KØN'], axis=1, inplace=True)

#Delete all rows that is not a municipality in the residence column
for val in ['Region', 'Province', 'All Denmark']: 
    I = hfudd10.BOPOMR.str.contains(val)
    hfudd10 = hfudd10.loc[I == False]
    
#Select all of the higher education categories sum them across each municipality and year.
group = hfudd10.loc[(hfudd10['HFUDD'] != 'Total')].copy()
group1 = group.groupby(['BOPOMR', 'TID'])['INDHOLD'].sum()
group2 = pd.DataFrame(group1)
group2.rename(columns = {'INDHOLD':'HIGH_EDU'}, inplace=True) #Rename column

#Select the total in each municipality and year.
total = hfudd10.loc[(hfudd10['HFUDD'] == 'Total')].copy()
total1 = total.groupby(['BOPOMR', 'TID'])['INDHOLD'].sum()
total2 = pd.DataFrame(total1)
total2.rename(columns = {'INDHOLD':'TOTAL'}, inplace=True) #Rename column

#Merge the data, reset index and get some proper variable names
merge = pd.merge(group2,total2,how='left',on=['BOPOMR','TID'])
merge['HIGH_EDU_SHARE'] = merge['HIGH_EDU'] / merge['TOTAL'] * 100
merge.reset_index(inplace=True)
merge.rename(columns = {'BOPOMR':'MUNICIPALITY','TID':'YEAR'}, inplace=True) #Rename column
merge

,MUNICIPALITY,YEAR,HIGH_EDU,TOTAL,HIGH_EDU_SHARE
0,Aabenraa,2006,1322,41865,3.157769
1,Aabenraa,2007,1367,41938,3.259574
2,Aabenraa,2008,1449,42097,3.442051
3,Aabenraa,2009,1498,42255,3.545143
4,Aabenraa,2010,1515,41920,3.614027
...,...,...,...,...,...
1381,Ærø,2015,195,4119,4.734159
1382,Ærø,2016,205,4089,5.013451
1383,Ærø,2017,211,3937,5.359411
1384,Ærø,2018,230,3887,5.917160
